In [2]:
# import packages
import pandas as pd

## Family Class

In [3]:
'''
FAMILY CLASS

family composition = no of adults, children, ages
dad_age == 0 means single parent family

'''

class Family():
    
    def __init__(self, mum_age, dad_age, num_of_children):
        
        self.mum_age = mum_age
        self.dad_age = dad_age
       
        self.num_of_adults = 1
        
        if self.dad_age >0:
            self.num_of_adults = 2
        
        #self.child_a_dob = child_a_dob
        self.num_of_children = num_of_children
        
    def __str__(self):
        
        return('Adults:' + str(self.num_of_adults) + 
               'Children:' + str(self.num_of_children))
        
        
    def adults_under_25(self):
        return self.mum_age < 25 and self.dad_age < 25
    

## Employment Class

In [4]:
'''
EMPLOYMENT CLASS
(to hold employment related methods - never instantiated)

CALCULATE INDIVIDUAL INCOME LEVEL
    a. tax thresholds and rates
         <= £1048pm = 0%
         £1048 to £3142pm = 20%
         > £3142 = 40% 
    b. NI 
         > £1048 paid at 8%

CALCULATE HOUSEHOLD INCOME LEVEL    
Input: 
    a. mum net income (use pt wage up to 30hpw, ft wage 30+hpw)
    b. dad net income
    
'''

class Employment():
    
    weeks_per_month = 365.25 / (12 * 7)
    
    def __init__(self, mum_hpw, mum_wage_pt, mum_wage_ft, dad_hpw, dad_wage):
        
        self.mum_hours = mum_hpw
        self.mum_wage_pt = mum_wage_pt
        self.mum_wage_ft = mum_wage_ft
        self.dad_hours = dad_hpw
        self.dad_wage = dad_wage

    
    # calculate net monthly income per person from hours worked per week and wage per hour
    def calculate_income(hpw, wage):
        
        # hours per week * weeks per month * wage per hour
        gross_monthly_income = (hpw * Employment.weeks_per_month) * wage
        
        # calculate tax and NI deductions
        ni = 0
        tax = 0
        
        if gross_monthly_income > 1048:
            ni = (gross_monthly_income-1048) * 0.08
            if gross_monthly_income <= 3142:
                tax = (gross_monthly_income-1048) * 0.2
            else:
                higher_rate = (gross_monthly_income-3142) * 0.4
                lower_rate = (gross_monthly_income-(gross_monthly_income-3142)-1048) * 0.2
                tax = higher_rate + lower_rate

        # return monthly income after tax and NI deductions (rounded to 2 dec)
        net_monthly_income = round(gross_monthly_income - (ni + tax), 2)
        
        return net_monthly_income

    
    # calculate monthly income for all adults in household
    def get_household_income(mum_hours, mum_wage_pt, mum_wage_ft, dad_hours, dad_wage):

        # under 30 hrs per week use pt wage, 30 hours + use ft wage
        mum_wage = mum_wage_pt
        if mum_hours >= 30:
            mum_wage = mum_wage_ft
        
        mum_income = Employment.calculate_income(mum_hours, mum_wage)
        dad_income = Employment.calculate_income(dad_hours, dad_wage)
        
        return mum_income + dad_income


## Region Class

In [5]:
'''
REGION CLASS

WAGES
median full-time wage for males (employment typically unaffected by parenthood)
25th percentile full-time and part-time wage for females

CHILDCARE COST CALCULATION:
cost per hour of wraparound childcare - wa_cph
cost per hour of holiday childcare - h_cph

baseline calculated on working a 35 hours week and then adjusted pro rata for part-time working. Per year:
- 28 days annual leave = £0
- 190 days wraparound, 3.5 hours per day at wa_cph
- 43 days holiday, 9 hours per day at h_cph

total cost / 1052 hours per year

= cost per hour

'''

class Region():

    def __init__(self, wa_cph, h_cph, male_wage_ft, female_wage_ft, female_wage_pt):
      
        self.male_wage_ft = male_wage_ft
        self.female_wage_ft = female_wage_ft
        self.female_wage_pt = female_wage_pt
        
        # storing wa and h childcare rates incase used in future for alternative working patterns
        self.wa_cph = wa_cph
        self.h_cph = h_cph
    
        wa_year_cost = wa_cph * 3.5 * 190
        h_year_cost = h_cph * 9 * 43
    
        self.childcare_cph = (wa_year_cost + h_year_cost) / 1052

    def __str__(self): 
        return('male_wage_ft: ' + str(self.male_wage_ft) + 
               ' female_wage_ft: ' + str(self.female_wage_ft) + 
               ' female_wage_pt: ' + str(self.female_wage_pt) + 
               ' childcare_cph: ' + str(self.childcare_cph))

### Set up regions

In [6]:
# create region objects and store in dictionary with region name key
region_object_dict = {'north_east' : Region(5.09, 3.38, 17.76, 13.23, 11.78),
                      'north_west' : Region(4.15, 3.24, 18.93, 13.53, 11.89),
                      'yorkshire_humber' : Region(4.32, 3.54, 18.33, 13.17, 11.75),
                      'east_midlands' : Region(4.74, 3.63, 18.64, 13.05, 11.67),
                      'west_midlands' : Region(6.33, 3.46, 18.73, 13.45, 11.89),
                      'east_england' : Region(4.79, 3.16, 20.63, 14.17, 11.97),
                      'outer_london' : Region(5.47, 3.56, 22.87, 15.7, 12.13),
                      'inner_london' : Region(8.94, 3.93, 25.04, 17.00, 12.08),
                      'south_east' : Region(5.87, 3.68, 21.71, 14.29, 11.99),
                      'south_west' : Region(4.21, 3.43, 19.21, 13.66, 11.97)
                     }

In [7]:
# trying out other comparisons
england_deciles_dict = {'england_10th_percentile' : Region(5.03, 3.48, 12.63, 12.20, 11.44),
                      'england' : Region(5.03, 3.48, 19.99, 13.95, 11.92)
                     }

In [8]:
# check this has worked
for key in region_object_dict:
    print(key,':',region_object_dict[key])

north_east : male_wage_ft: 17.76 female_wage_ft: 13.23 female_wage_pt: 11.78 childcare_cph: 4.460941064638783
north_west : male_wage_ft: 18.93 female_wage_ft: 13.53 female_wage_pt: 11.89 childcare_cph: 3.8152376425855516
yorkshire_humber : male_wage_ft: 18.33 female_wage_ft: 13.17 female_wage_pt: 11.75 childcare_cph: 4.033060836501901
east_midlands : male_wage_ft: 18.64 female_wage_ft: 13.05 female_wage_pt: 11.67 childcare_cph: 4.331663498098859
west_midlands : male_wage_ft: 18.73 female_wage_ft: 13.45 female_wage_pt: 11.89 childcare_cph: 5.274211026615969
east_england : male_wage_ft: 20.63 female_wage_ft: 14.17 female_wage_pt: 11.97 childcare_cph: 4.19037072243346
outer_london : male_wage_ft: 22.87 female_wage_ft: 15.7 female_wage_pt: 12.13 childcare_cph: 4.7673669201520905
inner_london : male_wage_ft: 25.04 female_wage_ft: 17.0 female_wage_pt: 12.08 childcare_cph: 7.0969676806083655
south_east : male_wage_ft: 21.71 female_wage_ft: 14.29 female_wage_pt: 11.99 childcare_cph: 5.06436311

In [9]:
# check how to access stored attributes
region_object_dict['west_midlands'].male_wage_ft

18.73

## Universal Credit Calculator

In [11]:
'''
UNIVERSAL CREDIT CALCULATOR (2024 rates)

1. calculate standard amount
    a. Single <25 - £311.68
    b. Single >=25 - £393.45
    c. Couple <25 - £489.23
    d. Couple >=25 - £617.60
    
    a. First child born < 6 April 2017 - £333.33
    b. First child born >= 6 April 2017 - £287.92
    c. Second child - £269.58
    
2. Minus income
    <=£673 per month = no deduction to UC
    For every £ >£673 = deduction of 55p (55%)
    
3. Up to benefit cap
    a. Outside Greater London - £1835 per month
    b. Inside Greater London - £2110.25 per month
    
    
Input:
    a. No. of adults
    b. Adults under 25
    c. Child 1 DOB
    d. Child 2 T/F
    e. Take home income per month
    f. Greater London T/F
    
'''

def calc_uc(family, income):

    
    # calculate standard amount for adults
    adults_under_25 = family.adults_under_25()
    
    
    if family.num_of_adults == 1:
        if adults_under_25:
            adult_payment = 311.68
        else:
            adult_payment = 393.45
            
    else:
        if adults_under_25:
            adult_payment = 489.23
        else:
            adult_payment = 617.60
    

    # calculate standard amount for children
    if family.num_of_children > 1:
        child_payment = 2 * 287.92
    else: child_payment = 287.92
    

    # calculate adjustment for income
    deduction = 0
    overpayment = income - 673 
    
    if overpayment > 0:
        deduction = overpayment * 0.55
        
 
    # subtotal before cap
    subtotal = round(adult_payment + child_payment - deduction, 2)
    if subtotal < 0:
        subtotal = 0
    
   
    # apply cap
    uc_payment = subtotal
   
    if subtotal > 1835:
        uc_payment = 1835
        

    return uc_payment

## Main function

In [13]:
'''
MAIN FUNCTION TO CALCULATE HOUSEHOLD MONTLYH INCOME AFTER CHILDCARE COSTS

1st loop = mum's working hours per week (15, 40, 5)
2nd loop = no. of children in household (1, 6)

Set dad's age to 0 for single parent family

Output list of 5 colomns of data (hpw), containing 5 rows (no. of children)

Childcare hours = (working hours / 2) assumes 1 hour travel between school and work x 2 (morning and afternoon)
i.e. 7 hour working day (9am-5pm, 1 hour lunch break) requires 3.5 hours childcare (8am-6pm)

FAMILY CONTRIBUTION TO CHILDCARE COSTS
If family eligible for UC (i.e. UC payment != 0) the UC covers 85% of childcare cost per month up to:
    a. £1014.63 for one child
    b. £1739.37 for 2+ children

'''

def get_monthly_household_income_data(mum_age, dad_age, mum_wage_pt, mum_wage_ft, dad_wage, childcare_cph):

    # if single parent family, dad attributes = 0
    if dad_age == 0:
        dad_hours = 0
    # if couple, dad works 38 hours per week (male median FT)
    else:
        dad_hours = 38

    # empty list to hold column data as lists
    list_of_column_data = []
    
    # loop to calculate income for increments of worked hours per week by mum
    for hours in range(0, 42, 7):
        income = Employment.get_household_income(hours, mum_wage_pt, mum_wage_ft, dad_hours, dad_wage)
        column = []

        # loop to generate family compositions based on number of children (between 1 and 5)
        for num_of_children in range(1,6):
            family = Family(mum_age, dad_age, num_of_children)

            
            # calculate UC payment based on family composition and income
            uc_payment = calc_uc(family, income)

            
            # calculate family requirement for childcare hours
            childcare_hours = 0
            
            if hours == 0:
                family_childcare_cost = 0
            else:
                childcare_hours = (hours / 2) * num_of_children * Employment.weeks_per_month

            
                # calculate total childcare cost
                total_childcare_cost = childcare_hours * childcare_cph
                uc_contribution = 0
    
                # if not eligible for UC, and household income under £100k, tax free childcare covers 20% of cost
                # up to £500 per child every 3 months
                if uc_payment == 0:
                    if income < 100000:
                        uc_contribution = total_childcare_cost * 0.2
                        tax_free_limit = (500/3) * num_of_children
                        if uc_contribution > tax_free_limit:
                            uc_contribution = tax_free_limit
                         
                # otherwise, UC covers 85% up to £1014.63 for one child or £1739.37 for 2+ children
                else: 
                    uc_contribution = total_childcare_cost * 0.85 
                    if num_of_children == 1 and uc_contribution > 1014.63:
                            uc_contribution = 1014.63
                        #family_childcare_cost = total_childcare_cost - uc_contribution
                    elif num_of_children > 1 and uc_contribution > 1739.37:
                            uc_contribution = 1739.37
                family_childcare_cost = total_childcare_cost - uc_contribution

            
            # income plus UC payment, less family contribution to childcare costs
            monthly_household_income_after_childcare = round((income + uc_payment) - family_childcare_cost, 2)

            # add values for different family compositions to column list
            column_data = monthly_household_income_after_childcare
            column.append(column_data)

        # add each column to parent list
        list_of_column_data.append(column)

    return list_of_column_data

## Test

In [14]:
test = get_monthly_household_income_data(22, 0, 11.44, 13.95, 0, 5.03)

In [15]:
print(test)

[[599.6, 887.52, 887.52, 887.52, 887.52], [936.32, 1212.75, 1201.27, 1189.79, 1178.31], [1260.16, 1525.12, 1502.15, 1479.19, 1456.22], [1405.38, 1658.85, 1624.41, 1589.96, 1555.51], [1507.14, 1749.13, 1703.2, 1657.27, 1611.34], [1515.81, 1962.74, 1905.33, 1847.92, 1790.51]]


#### Create dataframe from list data

In [16]:
df = pd.DataFrame(
        {'0 days' : test[0],
         '1 day' : test[1],
         '2 days' : test[2],
         '3 days' : test[3],
         '4 days' : test[4],
         '5 days' : test[5]
         }, index=pd.Index(['1 child', '2 children', '3 children', '4 children', '5 children'])
    )
df.style.format(precision=2).set_caption("Monthly household income after childcare costs based on days per week worked")

,0 days,1 day,2 days,3 days,4 days,5 days
1 child,599.60,936.32,1260.16,1405.38,1507.14,1515.81
2 children,887.52,1212.75,1525.12,1658.85,1749.13,1962.74
3 children,887.52,1201.27,1502.15,1624.41,1703.20,1905.33
4 children,887.52,1189.79,1479.19,1589.96,1657.27,1847.92
5 children,887.52,1178.31,1456.22,1555.51,1611.34,1790.51


#### Calculate change in monthly income per 5 hour increase in weekly worked hours

In [17]:
df_change = pd.DataFrame()
df_change['Increase to 1 day'] = df['1 day'] - df['0 days']
df_change['Increase to 2 days'] = df['2 days'] - df['1 day']
df_change['Increase to 3 days'] = df['3 days'] - df['2 days']
df_change['Increase to 4 days'] = df['4 days'] - df['3 days']
df_change['Increase to 5 days'] = df['5 days'] - df['4 days']

In [18]:
df_change.style.format(precision=2)\
.set_caption("Change in monthly household income after childcare costs with each extra day per week worked")

,Increase to 1 day,Increase to 2 days,Increase to 3 days,Increase to 4 days,Increase to 5 days
1 child,336.72,323.84,145.22,101.76,8.67
2 children,325.23,312.37,133.73,90.28,213.61
3 children,313.75,300.88,122.26,78.79,202.13
4 children,302.27,289.40,110.77,67.31,190.65
5 children,290.79,277.91,99.29,55.83,179.17


## Generate regional dataframes

In [19]:
'''
set dad's age to 0 for single parent family
UC allocation is higher if both parents over 25
'''
mum_age = 25
dad_age = 0

In [20]:
# run the main calculation for all regions using age values above
regional_income_data_dict = {}

for key in region_object_dict:
    regional_income_data_dict[key] = get_monthly_household_income_data(mum_age, dad_age, 
                                                                       region_object_dict[key].female_wage_pt,
                                                                       region_object_dict[key].female_wage_ft,
                                                                       region_object_dict[key].male_wage_ft, 
                                                                       region_object_dict[key].childcare_cph
    )

In [21]:
regional_income_data_dict

{'north_east': [[681.37, 969.29, 969.29, 969.29, 969.29],
  [1029.74, 1307.47, 1297.29, 1287.11, 1276.92],
  [1353.85, 1621.41, 1601.04, 1580.67, 1560.31],
  [1501.53, 1758.9, 1728.35, 1697.8, 1667.25],
  [1607.52, 1854.7, 1813.97, 1773.23, 1732.5],
  [1785.0, 2022.01, 1971.09, 1920.17, 1869.25]],
 'north_west': [[681.37, 969.29, 969.29, 969.29, 969.29],
  [1034.56, 1313.77, 1305.06, 1296.35, 1287.64],
  [1359.81, 1630.31, 1612.89, 1595.47, 1578.06],
  [1509.21, 1771.0, 1744.87, 1718.75, 1692.62],
  [1617.75, 1870.83, 1836.0, 1801.16, 1766.32],
  [1807.17, 2051.55, 2008.0, 1964.45, 1920.9]],
 'yorkshire_humber': [[681.37, 969.29, 969.29, 969.29, 969.29],
  [1029.8, 1308.52, 1299.31, 1290.1, 1280.9],
  [1354.99, 1624.49, 1606.08, 1587.67, 1569.25],
  [1503.57, 1763.87, 1736.25, 1708.63, 1681.01],
  [1610.24, 1861.34, 1824.51, 1787.68, 1750.86],
  [1786.93, 2028.81, 1982.78, 1936.75, 1890.71]],
 'east_midlands': [[681.37, 969.29, 969.29, 969.29, 969.29],
  [1026.69, 1304.72, 1294.83, 128

In [22]:
# create dictionary holding data as dataframe from list
regional_income_tables_dict = {}

for region in regional_income_data_dict:
    regional_income_tables_dict[region] = pd.DataFrame(
                {'0 days' : regional_income_data_dict[region][0],
                 '1 day' : regional_income_data_dict[region][1],
                 '2 days' : regional_income_data_dict[region][2],
                 '3 days' : regional_income_data_dict[region][3],
                 '4 days' : regional_income_data_dict[region][4],
                 '5 days' : regional_income_data_dict[region][5]
                 }, index=pd.Index(['1 child', '2 children', '3 children', '4 children', '5 children'])
            )

In [23]:
# format dataframe display
for region in regional_income_tables_dict:
    display(regional_income_tables_dict[region] \
            .style.format(precision=2).set_caption("Monthly household income after childcare costs (" + region + ")"))

,0 days,1 day,2 days,3 days,4 days,5 days
1 child,681.37,1029.74,1353.85,1501.53,1607.52,1785.00
2 children,969.29,1307.47,1621.41,1758.90,1854.70,2022.01
3 children,969.29,1297.29,1601.04,1728.35,1813.97,1971.09
4 children,969.29,1287.11,1580.67,1697.80,1773.23,1920.17
5 children,969.29,1276.92,1560.31,1667.25,1732.50,1869.25


,0 days,1 day,2 days,3 days,4 days,5 days
1 child,681.37,1034.56,1359.81,1509.21,1617.75,1807.17
2 children,969.29,1313.77,1630.31,1771.00,1870.83,2051.55
3 children,969.29,1305.06,1612.89,1744.87,1836.00,2008.00
4 children,969.29,1296.35,1595.47,1718.75,1801.16,1964.45
5 children,969.29,1287.64,1578.06,1692.62,1766.32,1920.90


,0 days,1 day,2 days,3 days,4 days,5 days
1 child,681.37,1029.80,1354.99,1503.57,1610.24,1786.93
2 children,969.29,1308.52,1624.49,1763.87,1861.34,2028.81
3 children,969.29,1299.31,1606.08,1736.25,1824.51,1982.78
4 children,969.29,1290.10,1587.67,1708.63,1787.68,1936.75
5 children,969.29,1280.90,1569.25,1681.01,1750.86,1890.71


,0 days,1 day,2 days,3 days,4 days,5 days
1 child,681.37,1026.69,1351.42,1499.16,1604.36,1777.61
2 children,969.29,1304.72,1619.57,1757.42,1852.72,2016.09
3 children,969.29,1294.83,1599.79,1727.75,1813.17,1966.64
4 children,969.29,1284.95,1580.01,1698.09,1773.62,1917.20
5 children,969.29,1275.06,1560.24,1668.42,1734.06,1867.76


,0 days,1 day,2 days,3 days,4 days,5 days
1 child,681.37,1031.23,1353.15,1499.22,1604.43,1786.57
2 children,969.29,1307.11,1616.99,1751.02,1844.19,2014.29
3 children,969.29,1295.07,1592.91,1714.90,1796.03,1954.09
4 children,969.29,1283.03,1568.83,1678.78,1747.87,1893.89
5 children,969.29,1270.99,1544.75,1642.66,1699.71,1833.69


,0 days,1 day,2 days,3 days,4 days,5 days
1 child,681.37,1036.14,1360.29,1509.00,1617.49,1834.44
2 children,969.29,1314.50,1629.08,1768.23,1867.14,2074.53
3 children,969.29,1304.93,1609.95,1739.53,1828.88,2026.70
4 children,969.29,1295.37,1590.81,1710.83,1790.62,1978.87
5 children,969.29,1285.80,1571.68,1682.13,1752.35,1931.04


,0 days,1 day,2 days,3 days,4 days,5 days
1 child,681.37,1039.70,1362.03,1509.79,1618.53,1723.56
2 children,969.29,1316.73,1628.19,1765.06,1862.92,2136.81
3 children,969.29,1305.85,1606.42,1732.41,1819.38,2082.39
4 children,969.29,1294.97,1584.66,1699.76,1775.85,2027.98
5 children,969.29,1284.08,1562.89,1667.11,1732.32,1973.56


,0 days,1 day,2 days,3 days,4 days,5 days
1 child,681.37,1032.86,1350.04,1492.36,1595.29,1724.19
2 children,969.29,1304.58,1605.56,1731.67,1818.40,2147.73
3 children,969.29,1288.38,1573.15,1683.07,1753.60,2066.72
4 children,969.29,1272.18,1540.75,1634.47,1688.79,1888.97
5 children,969.29,1255.97,1508.35,1585.86,1527.24,1348.94


,0 days,1 day,2 days,3 days,4 days,5 days
1 child,681.37,1034.76,1356.85,1503.61,1610.30,1830.39
2 children,969.29,1311.12,1621.65,1756.84,1851.97,2060.50
3 children,969.29,1299.56,1598.52,1722.16,1805.73,2002.70
4 children,969.29,1288.00,1575.40,1687.48,1759.48,1944.89
5 children,969.29,1276.44,1552.28,1652.80,1713.24,1887.09


,0 days,1 day,2 days,3 days,4 days,5 days
1 child,681.37,1036.75,1361.51,1510.83,1619.93,1812.34
2 children,969.29,1315.72,1631.52,1771.89,1872.03,2055.48
3 children,969.29,1306.76,1613.61,1745.02,1836.20,2010.71
4 children,969.29,1297.81,1595.70,1718.15,1800.38,1965.93
5 children,969.29,1288.85,1577.78,1691.29,1764.56,1921.15


In [24]:
# calculate change in monthly income with each increase of 5 working hours
regional_change_in_income_tables_dict = {}

for region in regional_income_tables_dict:
    df = regional_income_tables_dict[region]
    
    df_change = pd.DataFrame()
       
    df_change['Increase to 1 day'] = df['1 day'] - df['0 days']
    df_change['Increase to 2 days'] = df['2 days'] - df['1 day']
    df_change['Increase to 3 days'] = df['3 days'] - df['2 days']
    df_change['Increase to 4 days'] = df['4 days'] - df['3 days']
    df_change['Increase to 5 days'] = df['5 days'] - df['4 days']
    
    regional_change_in_income_tables_dict[region] = df_change


In [25]:
# earnings equal to £10 per hour
yellow = round(10*5*4.34524, 2)

# earnings equal to £5 per hour
orange = round(5*5*4.34524, 2)

# earnings equal to £0 per hour
red = 0

print('yellow: ' + str(yellow) + ', orange: ' + str(orange) + ', red: ' + str(red))

yellow: 217.26, orange: 108.63, red: 0


In [26]:
# format dataframe display - Change in monthly household income after childcare costs with each extra day per week worked
for region in regional_change_in_income_tables_dict:
    display(regional_change_in_income_tables_dict[region] \
            .style.format(precision=2).map(lambda x: 'background-color: #fff5b3' if x<yellow else 'background-color: #d9ffb3') \
            .map(lambda x: 'background-color: #ffd9b3' if x<orange else None) \
            .map(lambda x: 'background-color: #ffb3b3' if x<red else None) \
            .set_caption("Region: " + region ))
            

,Increase to 1 day,Increase to 2 days,Increase to 3 days,Increase to 4 days,Increase to 5 days
1 child,348.37,324.11,147.68,105.99,177.48
2 children,338.18,313.94,137.49,95.80,167.31
3 children,328.00,303.75,127.31,85.62,157.12
4 children,317.82,293.56,117.13,75.43,146.94
5 children,307.63,283.39,106.94,65.25,136.75


,Increase to 1 day,Increase to 2 days,Increase to 3 days,Increase to 4 days,Increase to 5 days
1 child,353.19,325.25,149.40,108.54,189.42
2 children,344.48,316.54,140.69,99.83,180.72
3 children,335.77,307.83,131.98,91.13,172.00
4 children,327.06,299.12,123.28,82.41,163.29
5 children,318.35,290.42,114.56,73.70,154.58


,Increase to 1 day,Increase to 2 days,Increase to 3 days,Increase to 4 days,Increase to 5 days
1 child,348.43,325.19,148.58,106.67,176.69
2 children,339.23,315.97,139.38,97.47,167.47
3 children,330.02,306.77,130.17,88.26,158.27
4 children,320.81,297.57,120.96,79.05,149.07
5 children,311.61,288.35,111.76,69.85,139.85


,Increase to 1 day,Increase to 2 days,Increase to 3 days,Increase to 4 days,Increase to 5 days
1 child,345.32,324.73,147.74,105.20,173.25
2 children,335.43,314.85,137.85,95.30,163.37
3 children,325.54,304.96,127.96,85.42,153.47
4 children,315.66,295.06,118.08,75.53,143.58
5 children,305.77,285.18,108.18,65.64,133.70


,Increase to 1 day,Increase to 2 days,Increase to 3 days,Increase to 4 days,Increase to 5 days
1 child,349.86,321.92,146.07,105.21,182.14
2 children,337.82,309.88,134.03,93.17,170.10
3 children,325.78,297.84,121.99,81.13,158.06
4 children,313.74,285.80,109.95,69.09,146.02
5 children,301.70,273.76,97.91,57.05,133.98


,Increase to 1 day,Increase to 2 days,Increase to 3 days,Increase to 4 days,Increase to 5 days
1 child,354.77,324.15,148.71,108.49,216.95
2 children,345.21,314.58,139.15,98.91,207.39
3 children,335.64,305.02,129.58,89.35,197.82
4 children,326.08,295.44,120.02,79.79,188.25
5 children,316.51,285.88,110.45,70.22,178.69


,Increase to 1 day,Increase to 2 days,Increase to 3 days,Increase to 4 days,Increase to 5 days
1 child,358.33,322.33,147.76,108.74,105.03
2 children,347.44,311.46,136.87,97.86,273.89
3 children,336.56,300.57,125.99,86.97,263.01
4 children,325.68,289.69,115.10,76.09,252.13
5 children,314.79,278.81,104.22,65.21,241.24


,Increase to 1 day,Increase to 2 days,Increase to 3 days,Increase to 4 days,Increase to 5 days
1 child,351.49,317.18,142.32,102.93,128.90
2 children,335.29,300.98,126.11,86.73,329.33
3 children,319.09,284.77,109.92,70.53,313.12
4 children,302.89,268.57,93.72,54.32,200.18
5 children,286.68,252.38,77.51,-58.62,-178.30


,Increase to 1 day,Increase to 2 days,Increase to 3 days,Increase to 4 days,Increase to 5 days
1 child,353.39,322.09,146.76,106.69,220.09
2 children,341.83,310.53,135.19,95.13,208.53
3 children,330.27,298.96,123.64,83.57,196.97
4 children,318.71,287.40,112.08,72.00,185.41
5 children,307.15,275.84,100.52,60.44,173.85


,Increase to 1 day,Increase to 2 days,Increase to 3 days,Increase to 4 days,Increase to 5 days
1 child,355.38,324.76,149.32,109.10,192.41
2 children,346.43,315.80,140.37,100.14,183.45
3 children,337.47,306.85,131.41,91.18,174.51
4 children,328.52,297.89,122.45,82.23,165.55
5 children,319.56,288.93,113.51,73.27,156.59
